训练zinc数据集的指令为
`python train_zinc.py --abs-pe rw --se khopgnn --gnn-type pna2 --dropout 0.2 --k-hop 3 --use-edge-attr`

In [ ]:

'''
Namespace(abs_pe='rw',
            abs_pe_dim=20,
            batch_norm=True,
            batch_size=128,
            crystal_dataset='ZINC',
            dim_hidden=64, dropout=0.2,
            edge_dim=32,
            epochs=2000,
            global_pool='mean',
            gnn_type='pna2',
            k_hop=3,
            layer_norm=False,
            lr=0.001,
            num_heads=8,
            num_layers=6,
            outdir='',
            save_logs=False,
            se='khopgnn',
            seed=0,
            use_cuda=True,
            use_edge_attr=True,
            warmup=5000,
            weight_decay=1e-05)
'''

在这个project中，train_zinc是可以运行的，请放心，接下来，我们来对比看看zinc数据和crystal数据的差别，为什么crystal经过转化之后就不可以运行

# import here

In [151]:
import importlib
from csat.data import GraphDataset

from torch_geometric.data import DataLoader
from torch_geometric import datasets
import csat.crytal_data
importlib.reload(csat.crytal_data)
from csat.crytal_data import crystal_graph_list
from csat.models import GraphTransformer
import os
import copy
import argparse
import numpy as np
import pandas as pd
from collections import defaultdict
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch_geometric.utils as utils

abs_pe='rw'
k_hop = 2
se = 'khopgnn'
use_edge_attr = True
abs_pe='rw'
abs_pe_dim=20
batch_norm=True
batch_size=128
dim_hidden=64
dropout=0.2
edge_dim=32
epochs=50
global_pool='mean'
gnn_type='pna2'
layer_norm=False
lr=0.001
num_heads=8
num_layers=6
outdir=''
save_logs=False
seed=0
use_cuda=True
epochs = 50
warmup=5000
weight_decay=1e-05
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 文件地址

In [167]:
zinc_data_path = './zinc_datasets/ZINC'
crystal_data_path = './sample-regression'

# ZINC and Crystal

In [169]:
train_zinc = datasets.ZINC(zinc_data_path, subset=True,split='train')

indices = range(0, 10) # 1~10
train_zinc = torch.utils.data.Subset(train_zinc, indices)



print(train_zinc,'zinc数据集是一个图的list集合。')
print(type(train_zinc))
print('zinc数据集图的数量：',len(train_zinc))
print('zinc数据集第1张图的数据类型为：',train_zinc[0])
print('zinc数据集第100张图的数据类型为：',train_zinc[9])

<torch.utils.data.dataset.Subset object at 0x000001F224052970> zinc数据集是一个图的list集合。
<class 'torch.utils.data.dataset.Subset'>
zinc数据集图的数量： 10
zinc数据集第1张图的数据类型为： Data(x=[29, 1], edge_index=[2, 64], edge_attr=[64], y=[1])
zinc数据集第100张图的数据类型为： Data(x=[18, 1], edge_index=[2, 38], edge_attr=[38], y=[1])


In [170]:
crystal_data = CIFData(crystal_data_path)
print(crystal_data)
print("crystal data 是一个元组的形式，其数据结构为(atom_fea, nbr_fea, nbr_fea_idx), target, cif_id")
print("对应的图数据类型为                             x,  edge_attr,   edge_index")
c_structures, target, cif_id = crystal_data[0]
print('atom_fea: (n_i, atom_fea_len)',c_structures[0].shape)
print('nbr_fea: (n_i, M, nbr_fea_len)',c_structures[1].shape)
print('nbr_fea_idx: (n_i, M)',c_structures[2].shape)
print("于是，我们需要把他转化为与zinc一样的graph_list")

crystal data 是一个元组的形式，其数据结构为(atom_fea, nbr_fea, nbr_fea_idx), target, cif_id
对应的图数据类型为                             x,  edge_attr,   edge_index
atom_fea: (n_i, atom_fea_len) torch.Size([8, 92])
nbr_fea: (n_i, M, nbr_fea_len) torch.Size([8, 12, 41])
nbr_fea_idx: (n_i, M) torch.Size([8, 12])
于是，我们需要把他转化为与zinc一样的graph_list


In [12]:
crystal_list = crystal_graph_list(crystal_data)
print('这是crystal list的数据类型',type(crystal_list))
print('crystal list图的数量：',len(crystal_list))
print('crystal list第1张图的数据类型为',crystal_list[0])
print('crystal list第10张图的数据类型为',crystal_list[9])

crystal data 是一个元组的形式，其数据结构为(atom_fea, nbr_fea, nbr_fea_idx), target, cif_id
对应的图数据类型为                             x,       
于是，我们需要把他转化为与zinc一样的graph_list
正在转化的数据集大小为： 10


100%|██████████| 10/10 [00:00<00:00, 43.00it/s]

这是crystal list的数据类型 <class 'list'>
crystal list图的数量： 10
crystal list第1张图的数据类型为 Data(x=[8, 92], edge_index=[8, 12], edge_attr=[8, 12, 41], y=[1])
crystal list第10张图的数据类型为 Data(x=[8, 92], edge_index=[8, 12], edge_attr=[8, 12, 41], y=[1])


由此我们可以发现

zinc：


zinc数据集第1张图的数据类型为： Data(x=[29, 1], edge_index=[2, 64], edge_attr=[64], y=[1])

zinc数据集第100张图的数据类型为： Data(x=[27, 1], edge_index=[2, 60], edge_attr=[60], y=[1])

crystal：

crystal list第1张图的数据类型为 Data(x=[8, 92], edge_index=[8, 12], edge_attr=[8, 12, 41], y=[1])

crystal list第10张图的数据类型为 Data(x=[8, 92], edge_index=[8, 12], edge_attr=[8, 12, 41],y=[1])

在CIFDATA数据中
```text
atom_fea: torch.Tensor shape (n_i, atom_fea_len)
nbr_fea: torch.Tensor shape (n_i, M, nbr_fea_len)
nbr_fea_idx: torch.LongTensor shape (n_i, M)
target: torch.Tensor shape (1, )
```


我们就需要对其做出修改,将egde_index改为[2, num_edges],将edge_attr降维为1维度。

In [26]:
print('这是重新编写之后的crystal list！')
crystal_list = crystal_graph_list(crystal_data)
print('这是crystal list的数据类型',type(crystal_list))
print('crystal list图的数量：',len(crystal_list))
print('crystal list第1张图的数据类型为',crystal_list[0])
print('crystal list第10张图的数据类型为',crystal_list[9])

这是重新编写之后的crystal list！
正在转化的数据集大小为： 10


100%|██████████| 10/10 [00:00<00:00, 5024.32it/s]

这是crystal list的数据类型 <class 'list'>
crystal list图的数量： 10
crystal list第1张图的数据类型为 Data(x=[8, 92], edge_index=[2, 48], edge_attr=[96], y=[1])
crystal list第10张图的数据类型为 Data(x=[8, 92], edge_index=[2, 48], edge_attr=[96], y=[1])


然后调用sat的graphdataset提取子图

In [97]:
train_zinc_dset = GraphDataset(train_zinc,degree=True, k_hop=k_hop, se=se,use_subgraph_edge_attr=use_edge_attr)
print(train_zinc_dset)
print('含有子图第一张图数据为：',train_zinc_dset[0])

Extracting 2-hop subgraphs...
Done!
含有子图第一张图数据为： Data(x=[29], edge_index=[2, 64], edge_attr=[64], y=[1, 1], complete_edge_index=[2, 841], degree=[29], subgraph_edge_index=[2, 326], num_subgraph_nodes=187, subgraph_node_idx=[187], subgraph_edge_attr=[326], subgraph_indicator=[187])


In [163]:
train_c_dset = GraphDataset(crystal_list,degree=True, k_hop=k_hop, se=se,use_subgraph_edge_attr=use_edge_attr)
print('含有子图的crystal数据集第一张图数据为：',trian_c_dset[0])

Extracting 2-hop subgraphs...
Done!
含有子图的crystal数据集第一张图数据为： Data(x=[8, 92], edge_index=[2, 12], edge_attr=[12], y=[1, 1], complete_edge_index=[2, 64], subgraph_edge_index=[2, 14], num_subgraph_nodes=15, subgraph_node_idx=[15], subgraph_edge_attr=[14], subgraph_indicator=[15], degree=[8])


我的原数据为`Data(x=[8, 92], edge_index=[8, 12], edge_attr=[8, 12, 41], y=[1])`，再经过edge_index转化[2,n]以及edge_attr降维之后变成：`Data(x=[8, 92], edge_index=[2, 48], edge_attr=[96], y=[1])`。

于是出现了问题：edge_mask 是一个布尔张量，用于选择与当前子图相关的边。它的长度应该等于graph.edge_index 的边数（即 graph.edge_index.shape[1]）。但是graph.edge_attr 的长度是 96，因此无法用 edge_mask 对 graph.edge_attr 进行索引。

因此对crystal_graph_list进行改写，使用了稀疏矩阵的方法重新改写，再测试

In [171]:
print('这是第二次重新编写之后的crystal list！')
crystal_list = crystal_graph_list(crystal_data)
print('这是crystal list的数据类型',type(crystal_list))
print('crystal list图的数量：',len(crystal_list))
print('crystal list第1张图的数据类型为',crystal_list[0])
print('crystal list第10张图的数据类型为',crystal_list[9])

这是第二次重新编写之后的crystal list！
正在转化的数据集大小为： 10


  0%|          | 0/10 [00:00<?, ?it/s]E:\WYX_Project\SAT-CGCN-Demo-1\csat\crytal_data.py:342: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.long)
E:\WYX_Project\SAT-CGCN-Demo-1\csat\crytal_data.py:343: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr = torch.tensor(edge_attr, dtype=torch.long)
E:\WYX_Project\SAT-CGCN-Demo-1\csat\crytal_data.py:344: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target, dtype=torch.long)
E:\anaconda3\envs\pytorch\lib\site-packages\pymatgen\io\cif.

这是crystal list的数据类型 <class 'list'>
crystal list图的数量： 10
crystal list第1张图的数据类型为 Data(x=[8, 92], edge_index=[2, 12], edge_attr=[12], y=[1])
crystal list第10张图的数据类型为 Data(x=[8, 92], edge_index=[2, 12], edge_attr=[12], y=[1])


结果表明很成功

In [172]:
train_c_dset = GraphDataset(crystal_list,degree=True, k_hop=k_hop, se=se,use_subgraph_edge_attr=use_edge_attr)
print('c',trian_c_dset[0])

train_zinc_dset = GraphDataset(train_zinc, degree=True, k_hop=k_hop, se=se, use_subgraph_edge_attr=use_edge_attr)
print(train_zinc_dset)
print('z', train_zinc_dset[0])

Extracting 2-hop subgraphs...
Done!
c Data(x=[8, 92], edge_index=[2, 12], edge_attr=[12], y=[1, 1], complete_edge_index=[2, 64], subgraph_edge_index=[2, 14], num_subgraph_nodes=15, subgraph_node_idx=[15], subgraph_edge_attr=[14], subgraph_indicator=[15], degree=[8])
Extracting 2-hop subgraphs...
Done!
z Data(x=[29], edge_index=[2, 64], edge_attr=[64], y=[1, 1], complete_edge_index=[2, 841], degree=[29], subgraph_edge_index=[2, 326], num_subgraph_nodes=187, subgraph_node_idx=[187], subgraph_edge_attr=[326], subgraph_indicator=[187])


在开始训练之前，出现了如下：

RuntimeError: The expanded size of the tensor (276) must match the existing size (3) at non-singleton dimension 0.  Target sizes: [276, 64].  Tensor sizes: [3, 1]

In [165]:
print('zinc数据集第1张图的数据类型为：',train_zinc[0])
print('\n')
print('含有子图的zinc第1张',train_zinc_dset[0])
print('\n')
print('zinc数据集在loader前',train_zinc_dset[0])
train_zinc_loader = DataLoader(train_zinc_dset, batch_size=128,shuffle=True)
print('\n')
for batch in train_zinc_loader:
    print('loader之后：',batch)
    break

zinc数据集第1张图的数据类型为： Data(x=[29, 1], edge_index=[2, 64], edge_attr=[64], y=[1])


含有子图的zinc第1张 Data(x=[29], edge_index=[2, 64], edge_attr=[64], y=[1, 1], complete_edge_index=[2, 841], degree=[29], subgraph_edge_index=[2, 326], num_subgraph_nodes=187, subgraph_node_idx=[187], subgraph_edge_attr=[326], subgraph_indicator=[187])


zinc数据集在loader前 Data(x=[29], edge_index=[2, 64], edge_attr=[64], y=[1, 1], complete_edge_index=[2, 841], degree=[29], subgraph_edge_index=[2, 326], num_subgraph_nodes=187, subgraph_node_idx=[187], subgraph_edge_attr=[326], subgraph_indicator=[187])


loader之后： DataBatch(x=[222], edge_index=[2, 472], edge_attr=[472], y=[10, 1], complete_edge_index=[2, 5152], degree=[222], subgraph_edge_index=[2, 2310], num_subgraph_nodes=[10], subgraph_node_idx=[1334], subgraph_edge_attr=[2310], subgraph_indicator=[1334], batch=[222], ptr=[11])


In [173]:
print('c数据集第1张图的数据类型为：',crystal_list[0])
print('\n')
print('含有子图的c第1张',train_c_dset[0])
print('\n')
print('crystal数据集在loader前',train_c_dset[0])
print('\n')

train_c_loader = DataLoader(train_c_dset, batch_size=128, shuffle=True)
for batch in train_c_loader:
    print('loader之后：',batch)
    break

c数据集第1张图的数据类型为： Data(x=[8, 92], edge_index=[2, 12], edge_attr=[12], y=[1])


含有子图的c第1张 Data(x=[8, 92], edge_index=[2, 12], edge_attr=[12], y=[1, 1], complete_edge_index=[2, 64], degree=[8], subgraph_edge_index=[2, 14], num_subgraph_nodes=15, subgraph_node_idx=[15], subgraph_edge_attr=[14], subgraph_indicator=[15])


crystal数据集在loader前 Data(x=[8, 92], edge_index=[2, 12], edge_attr=[12], y=[1, 1], complete_edge_index=[2, 64], subgraph_edge_index=[2, 14], num_subgraph_nodes=15, subgraph_node_idx=[15], subgraph_edge_attr=[14], subgraph_indicator=[15], degree=[8])


loader之后： DataBatch(x=[65, 92], edge_index=[2, 120], edge_attr=[120], y=[10, 1], complete_edge_index=[2, 513], degree=[65], subgraph_edge_index=[2, 129], num_subgraph_nodes=[10], subgraph_node_idx=[100], subgraph_edge_attr=[129], subgraph_indicator=[100], batch=[65], ptr=[11])


从数据上看，没有什么问题

In [135]:
deg = torch.cat([
        utils.degree(data.edge_index[1], num_nodes=data.num_nodes) for
        data in train_zinc_dset])
zinc_model = GraphTransformer(in_size=28,
                             num_class=1,
                             d_model=dim_hidden,
                             dim_feedforward=2 * dim_hidden,
                             dropout=dropout,
                             num_heads=num_heads,
                             num_layers=num_layers,
                             batch_norm=batch_norm,
                             abs_pe=abs_pe,
                             abs_pe_dim=abs_pe_dim,
                             gnn_type=gnn_type,
                             use_edge_attr=use_edge_attr,
                             num_edge_features=4,
                             edge_dim=edge_dim,
                             k_hop=k_hop,
                             se=se,
                             deg=deg,
                             global_pool=global_pool)

在这里需要注意一个问题

zinc数据集中

number of node attributes for ZINC crystal_dataset

n_tags = 28------------>in_size

num_edge_features = 4

而我们crystal数据集的这两个参数是未知的，于是我使用92，41代替


In [136]:
deg = torch.cat([
        utils.degree(data.edge_index[1], num_nodes=data.num_nodes) for
        data in train_c_dset])
c_model = GraphTransformer(in_size=92,
                             num_class=1,
                             d_model=dim_hidden,
                             dim_feedforward=2 * dim_hidden,
                             dropout=dropout,
                             num_heads=num_heads,
                             num_layers=num_layers,
                             batch_norm=batch_norm,
                             abs_pe=abs_pe,
                             abs_pe_dim=abs_pe_dim,
                             gnn_type=gnn_type,
                             use_edge_attr=use_edge_attr,
                             num_edge_features=4,
                             edge_dim=edge_dim,
                             k_hop=k_hop,
                             se=se,
                             deg=deg,
                             global_pool=global_pool)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001F261218160>
Traceback (most recent call last):
  File "E:\anaconda3\envs\pytorch\lib\site-packages\torch\utils\data\dataloader.py", line 1477, in __del__
    self._shutdown_workers()
  File "E:\anaconda3\envs\pytorch\lib\site-packages\torch\utils\data\dataloader.py", line 1435, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


In [156]:
def train_out(loader, model,optimizer,abs_pe):
    for i, data in enumerate(loader):
        print(data)
        size = len(data.y)
        print(abs_pe)
        if abs_pe == 'lap':
            sign_flip = torch.rand(data.abs_pe.shape[-1])
            sign_flip[sign_flip >= 0.5] = 1.0
            sign_flip[sign_flip < 0.5] = -1.0
            data.abs_pe = data.abs_pe * sign_flip.unsqueeze(0)


        if use_cuda:
            data = data.cuda()

        optimizer.zero_grad()
        output = model(data)
        return output

In [139]:
zinc_model = zinc_model.to(device)
optimizer = optim.AdamW(zinc_model.parameters(), lr=lr, weight_decay=weight_decay)
zinc_out = train_out(train_zinc_loader, zinc_model,optimizer,abs_pe)


DataBatch(x=[222], edge_index=[2, 472], edge_attr=[472], y=[10, 1], complete_edge_index=[2, 5152], degree=[222], subgraph_edge_index=[2, 2310], num_subgraph_nodes=[10], subgraph_node_idx=[1334], subgraph_edge_attr=[2310], subgraph_indicator=[1334], batch=[222], ptr=[11])
rw


In [158]:
c_model = zinc_model.to(device)
optimizer = optim.AdamW(zinc_model.parameters(), lr=lr, weight_decay=weight_decay)
c_out = train_out(train_c_loader, zinc_model,optimizer,abs_pe)

DataBatch(x=[65, 92], edge_index=[2, 120], edge_attr=[120], y=[10, 1], complete_edge_index=[2, 513], subgraph_edge_index=[2, 129], num_subgraph_nodes=[10], subgraph_node_idx=[100], subgraph_edge_attr=[129], subgraph_indicator=[100], degree=[65], batch=[65], ptr=[11])
rw


RuntimeError: The expanded size of the tensor (9200) must match the existing size (100) at non-singleton dimension 0.  Target sizes: [9200, 64].  Tensor sizes: [100, 1]